<a href="https://colab.research.google.com/github/jorgewellison/classifiers-tcc/blob/main/Pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação das bibliotecas**

In [140]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

In [3]:
pd.set_option('display.max_rows', 50)

**Tratamento dos indicadores do IDHM**

In [141]:
atlas = pd.read_excel("/content/drive/My Drive/Colab Notebooks/covid-idhm/atlas2013_dadosbrutos_pt.xlsx", sheet_name=1, 
          usecols=['ANO', 'UF', 'Codmun7', 'Município', 'ESPVIDA', 'FECTOT', 'MORT1', 'SOBRE60', 'T_ENV', 'E_ANOSESTUDO', 'T_ANALF11A14', 'T_ANALF25M', 'T_ATRASO_0_BASICO', 'T_ATRASO_2_BASICO', 'T_FLPRE', 'T_FLSUPER', 'T_FREQSUPER1517', 'T_SUPER25M', 'T_MED25M', 'GINI', 'PIND', 'PINDCRI', 'PMPOB', 'PMPOBCRI', 'RDPC', 'CPR', 'EMP', 'P_AGRO', 'P_COM', 'P_CONSTR', 'P_EXTR', 'P_FORMAL', 'P_FUND', 'P_MED', 'P_SERV', 'P_SIUP', 'P_SUPER', 'T_DES2529', 'TRABCC', 'TRABPUB', 'TRABSC', 'T_AGUA', 'T_BANAGUA', 'T_DENS', 'T_LIXO', 'T_LUZ', 'AGUA_ESGOTO', 'PAREDE', 'T_CRIFUNDIN_TODOS', 'T_FORA4A5', 'T_FORA6A14', 'T_FUNDIN_TODOS', 'T_FUNDIN_TODOS_MMEIO', 'T_FUNDIN18MINF', 'T_M10A14CF', 'T_M15A17CF', 'T_MULCHEFEFIF014', 'T_NESTUDA_NTRAB_MMEIO', 'T_OCUPDESLOC_1', 'T_RMAXIDOSO', 'T_SLUZ', 'HOMEMTOT', 'MULHERTOT', 'pesoRUR', 'pesotot', 'pesourb', 'POP', 'IDHM', 'IDHM_E', 'IDHM_L', 'IDHM_R'], 
          dtype={'UF': str}).rename(columns={'ESPVIDA': 'expec_vida', 'FECTOT': 'taxa_fecund', 'SOBRE60': '%sobreviver_60a', 'T_ENV': 'taxa_envelh', 'E_ANOSESTUDO': 'expec_anos_estudo_18'})
atlasbr = atlas.loc[(atlas['ANO'] == 2010)]

In [142]:
atlasbr.head()

,ANO,UF,Codmun7,Município,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R
11130,2010,11,1100015,ALTA FLORESTA D'OESTE,70.75,2.24,23.8,83.18,5.84,8.67,1.22,15.76,57.62,22.03,34.15,10.50,2.73,19.19,5.51,0.58,14.29,19.01,26.04,37.33,476.99,34.73,2.83,38.56,13.78,5.31,0.00,41.68,45.49,31.05,33.18,0.69,7.72,5.06,25.17,5.31,23.17,93.69,80.20,22.58,94.05,93.98,9.31,7.18,45.73,60.82,4.32,40.27,24.81,54.06,0.00,6.75,14.51,16.12,1.15,2.20,6.02,12656,11736,10422,24392,13970,24097,0.641,0.526,0.763,0.657
11131,2010,11,1100023,ARIQUEMES,73.36,1.91,19.2,87.69,4.36,9.18,1.55,10.48,58.43,19.35,40.96,12.91,0.18,31.45,8.13,0.53,4.36,6.65,11.54,18.76,689.95,27.66,2.30,14.97,17.99,8.68,3.32,52.81,57.26,39.76,40.06,0.77,9.29,5.85,41.15,6.30,19.21,98.54,85.35,27.15,96.72,98.58,11.40,5.62,29.58,44.66,2.79,26.59,12.27,37.81,0.00,9.10,18.07,9.67,0.16,1.40,1.42,45543,44810,13828,90353,76525,89766,0.702,0.600,0.806,0.716
11132,2010,11,1100031,CABIXI,70.39,2.15,24.8,82.57,6.87,10.09,0.62,16.78,66.22,15.59,58.34,9.86,1.05,20.42,4.93,0.51,7.27,8.51,21.20,31.01,457.17,32.98,0.67,45.49,6.46,3.62,0.00,39.38,44.10,30.24,36.91,0.52,6.25,3.60,24.03,7.79,22.12,95.49,91.63,19.93,99.14,96.36,3.31,0.84,41.25,33.08,1.97,38.52,21.53,54.79,1.12,5.47,7.03,16.59,0.65,3.10,3.64,3266,3047,3620,6313,2693,6303,0.650,0.559,0.757,0.650
11133,2010,11,1100049,CACOAL,74.27,1.90,14.3,88.56,5.57,9.74,1.01,10.95,66.69,15.57,51.33,17.45,0.44,28.06,8.71,0.57,5.97,8.77,13.08,20.03,738.06,24.97,2.09,19.69,19.00,7.16,0.25,55.07,55.78,37.67,39.07,0.85,10.29,6.22,40.10,7.70,18.21,97.96,93.44,20.50,98.17,98.89,1.85,3.39,25.72,37.59,2.39,26.16,11.58,39.87,0.68,6.03,12.80,9.64,0.43,1.84,1.11,39124,39450,16653,78574,61921,78141,0.718,0.620,0.821,0.727
11134,2010,11,1100056,CEREJEIRAS,72.94,2.12,18.1,86.72,6.44,10.22,1.14,13.51,72.15,12.93,49.74,10.77,2.02,24.06,7.09,0.50,4.72,6.93,13.70,22.08,577.18,28.46,2.36,26.67,14.08,8.60,0.06,45.54,49.50,34.10,38.39,0.57,8.53,7.22,24.74,9.77,25.05,97.53,86.87,16.66,91.89,98.85,5.34,0.85,35.81,38.73,1.46,32.21,15.53,46.96,0.00,1.77,25.88,10.16,0.38,1.43,1.15,8551,8478,2610,17029,14419,16942,0.692,0.602,0.799,0.688


**Substituição dos valores nulos**

In [143]:
base = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/covid19-setembro.csv") 
base.head()

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
0,40,2020-09-27,183,AC,Acrelândia,1200013.0,city,468,3067.64552,0,9,0,0.0192,15256.0,False,True
1,40,2020-09-27,150,AC,Assis Brasil,1200054.0,city,617,8318.72725,0,9,0,0.0146,7417.0,False,True
2,40,2020-09-27,142,AC,Brasiléia,1200104.0,city,1165,4433.36631,0,20,0,0.0172,26278.0,False,True
3,40,2020-09-27,173,AC,Bujari,1200138.0,city,374,3643.09371,0,7,0,0.0187,10266.0,False,True
4,40,2020-09-27,143,AC,Capixaba,1200179.0,city,250,2130.74235,0,7,0,0.0280,11733.0,False,True


In [144]:
base.isnull().sum()

epidemiological_week                              0
date                                              0
order_for_place                                   0
state                                             0
city                                              0
city_ibge_code                                   19
place_type                                        0
last_available_confirmed                          0
last_available_confirmed_per_100k_inhabitants    22
new_confirmed                                     0
last_available_deaths                             0
new_deaths                                        0
last_available_death_rate                         0
estimated_population_2019                        19
is_last                                           0
is_repeated                                       0
dtype: int64

In [145]:
pd.isnull(base['last_available_confirmed_per_100k_inhabitants'])
base.loc[pd.isnull(base['last_available_confirmed_per_100k_inhabitants'])]

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
57,40,2020-09-27,183,AL,Importados/Indefinidos,NaN,city,8,NaN,0,8,0,1.0000,NaN,False,True
367,40,2020-09-27,188,BA,Importados/Indefinidos,NaN,city,4804,NaN,-703,47,0,0.0098,NaN,False,False
698,40,2020-09-27,193,CE,Importados/Indefinidos,NaN,city,2105,NaN,-6,0,0,0.0000,NaN,False,False
807,40,2020-09-27,186,DF,Importados/Indefinidos,NaN,city,23203,NaN,88,262,1,0.0113,NaN,True,False
841,40,2020-09-27,188,ES,Importados/Indefinidos,NaN,city,1158,NaN,0,23,0,0.0199,NaN,False,True
994,40,2020-09-27,193,GO,Importados/Indefinidos,NaN,city,0,NaN,0,0,0,0.0000,NaN,True,False
1695,40,2020-09-27,191,MG,Importados/Indefinidos,NaN,city,6995,NaN,2269,229,47,0.0327,NaN,True,False
2326,40,2020-09-27,175,MT,Importados/Indefinidos,NaN,city,284,NaN,0,37,0,0.1303,NaN,False,True
2473,40,2020-09-27,161,PA,Importados/Indefinidos,NaN,city,0,NaN,0,0,0,0.0000,NaN,False,False
2650,40,2020-09-27,102,PB,Importados/Indefinidos,NaN,city,0,NaN,0,0,0,0.0000,NaN,False,False


In [146]:
base = base.drop([57, 367, 698, 807, 841, 994, 1695, 2326, 2473, 2650, 2865, 3346, 3626, 3740, 3879, 3918, 4131, 4539, 5032])

**Remodelando o dataframe dos casos de covid**

In [147]:
cols = list(base.columns.values)
df = base[['city_ibge_code', 'estimated_population_2019', 'last_available_confirmed_per_100k_inhabitants']]
df.fillna(0,inplace=True)
df.isnull().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


city_ibge_code                                   0
estimated_population_2019                        0
last_available_confirmed_per_100k_inhabitants    0
dtype: int64

In [148]:
base_mort = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/cases-brazil-cities.csv", usecols=['ibgeID', 'deaths_per_100k_inhabitants'])
base_mort.head()

,ibgeID,deaths_per_100k_inhabitants
0,5200050,79.79027
1,3100104,14.30820
2,5200100,54.88474
3,1500107,71.02183
4,3100203,8.60696


**Junção da tabela de métricas de indicadores do IDHM (Atlas) + dataFrame covid**

In [149]:
idhmcovid = pd.merge(atlasbr, df, left_on="Codmun7", right_on="city_ibge_code", how="left")
idhmcovid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5564
Data columns (total 74 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   ANO                                            5565 non-null   int64  
 1   UF                                             5565 non-null   object 
 2   Codmun7                                        5565 non-null   int64  
 3   Município                                      5565 non-null   object 
 4   expec_vida                                     5565 non-null   float64
 5   taxa_fecund                                    5565 non-null   float64
 6   MORT1                                          5565 non-null   float64
 7   %sobreviver_60a                                5565 non-null   float64
 8   taxa_envelh                                    5565 non-null   float64
 9   expec_anos_estudo_18                           5565 

In [150]:
idhmcovid.loc[pd.isnull(idhmcovid['last_available_confirmed_per_100k_inhabitants'])]

,ANO,UF,Codmun7,Município,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R,city_ibge_code,estimated_population_2019,last_available_confirmed_per_100k_inhabitants
1086,2010,24,2401305,AUGUSTO SEVERO,71.46,2.19,22.5,84.32,9.48,8.84,10.01,35.44,52.94,27.41,47.49,5.38,1.02,18.81,3.53,0.68,23.56,33.46,41.22,56.81,388.39,27.76,0.63,43.60,9.71,5.21,0.73,23.69,40.88,29.77,33.45,0.60,5.24,8.04,17.11,2.15,32.85,55.03,40.84,30.72,91.93,98.32,27.06,9.59,49.30,14.59,1.99,42.64,32.56,64.75,1.10,5.06,20.76,21.79,2.60,5.16,1.68,4708,4581,4287,9289,5002,9258,0.621,0.495,0.774,0.624,NaN,NaN,NaN
2333,2010,31,3108503,Botumirim,71.94,3.22,20.5,77.83,6.45,8.78,1.89,23.13,55.68,23.11,48.83,6.18,0.00,15.09,3.07,0.46,20.02,31.68,39.74,57.58,230.25,14.38,0.67,56.94,5.26,7.24,0.51,36.34,34.45,21.52,25.82,0.99,4.11,9.10,18.96,14.63,23.66,80.32,67.45,28.89,74.92,96.47,6.75,1.25,36.25,48.27,3.48,34.71,24.14,56.01,0.63,3.72,22.38,25.18,2.06,4.27,3.53,3395,3102,3027,6497,3470,6478,0.602,0.517,0.782,0.540,NaN,NaN,NaN
2355,2010,31,3110400,Camacho,75.38,1.49,14.8,83.07,9.29,9.91,2.00,16.35,70.33,9.29,56.07,16.66,6.11,19.33,4.62,0.46,3.14,5.01,20.08,37.42,425.80,23.94,1.77,62.02,5.01,1.52,0.34,30.48,46.17,33.18,24.55,0.22,6.27,1.94,12.87,14.09,45.14,86.54,99.54,6.53,98.53,99.94,0.00,0.23,30.21,31.41,1.14,31.67,14.31,56.15,0.00,0.00,4.20,15.39,0.63,1.29,0.06,1665,1489,1696,3154,1458,3151,0.690,0.612,0.840,0.639,NaN,NaN,NaN
2414,2010,31,3115607,Cedro do Abaeté,73.04,2.17,18.5,79.55,12.89,9.83,0.00,16.97,67.43,12.75,81.21,18.36,0.00,17.79,5.40,0.39,2.08,3.11,11.92,21.79,431.20,17.39,0.50,40.96,5.26,0.95,1.64,35.08,42.78,26.79,44.41,0.00,9.46,14.07,13.02,19.07,35.11,95.51,98.46,14.82,90.96,99.82,0.52,0.00,24.41,0.00,0.00,34.60,10.23,54.71,0.00,0.00,16.04,14.74,0.50,3.41,0.18,628,582,177,1210,1033,1210,0.678,0.608,0.801,0.641,NaN,NaN,NaN
2819,2010,31,3149408,Pedro Teixeira,72.61,1.72,19.3,78.88,8.35,8.83,0.80,13.83,63.86,18.91,33.90,6.64,0.00,16.13,4.98,0.41,6.89,11.52,24.67,43.47,309.25,31.53,0.00,46.06,4.15,7.11,0.40,48.48,39.70,24.41,32.16,0.43,7.14,0.00,14.79,23.26,24.76,92.83,93.96,19.44,100.00,98.69,5.02,0.26,42.32,44.59,2.10,35.92,22.68,52.29,0.00,2.76,37.32,26.88,2.07,4.04,1.31,948,837,820,1785,965,1785,0.637,0.555,0.794,0.587,NaN,NaN,NaN
2869,2010,31,3153806,Queluzito,74.39,1.39,16.3,81.60,12.84,8.55,0.00,8.08,60.20,20.33,74.44,7.52,0.00,16.71,4.39,0.46,2.78,3.73,8.58,18.59,513.72,34.84,1.01,34.16,8.75,5.86,0.68,48.63,48.38,26.55,36.77,0.00,5.83,7.31,25.13,10.27,24.14,81.89,99.42,10.10,99.81,100.00,0.48,0.00,25.94,11.56,0.72,33.11,12.48,52.72,0.00,0.00,28.95,11.04,0.70,3.92,0.00,957,904,1014,1861,847,1861,0.682,0.575,0.823,0.669,NaN,NaN,NaN
3007,2010,31,3165206,São Thomé das Letras,73.03,2.37,18.5,79.54,5.65,9.25,1.28,16.10,60.02,19.58,56.58,4.69,1.90,20.74,5.06,0.40,2.45,3.62,13.48,22.23,428.33,24.84,1.75,23.58,8.72,3.06,28.19,48.34,45.36,26.87,25.52,0.49,5.43,5.22,35.45,6.68,23.61,84.51,98.07,24.72,100.00,98.36,0.48,1.14,39.97,25.10,1.96,37.22,19.01,41.77,0.00,1.69,24.20,8.86,1.22,1.19,1.64,3539,3116,2798,6655,3857,6576,0.667,0.578,0.801,0.640,NaN,NaN,NaN
3082,2010,31,3171071,Veredinha,72.11,2.11,20.2,78.10,7.33,9.15,1.15,30.06,63.00,17.79,61.62,8.99,0.00,16.11,4.31,0.43,14.66,21.39,29.45,43.46,293.08,10.46,0.62,47.72,9.40,10.15,0.39,34.75,36.62,23.11,25.54,1.15,4.92,13.45,22.44,10.00,25.94,76.96,74.82,17.64,94.88,98.21,15.23,0.06,28.99,28.54,2.01,31.85,18.43,

**Calculando média de coluna que serão utilizadas**

In [151]:
casosMedios = idhmcovid['last_available_confirmed_per_100k_inhabitants'].mean()
casosMedios = math.floor(casosMedios)

populacaoMedia = idhmcovid['estimated_population_2019'].mean()
populacaoMedia = math.floor(populacaoMedia)




**Trocando valores nulos pela média**

In [152]:
idhmcovid.update(idhmcovid['last_available_confirmed_per_100k_inhabitants'].fillna(casosMedios))
idhmcovid.update(idhmcovid['estimated_population_2019'].fillna(populacaoMedia))
idhmcovid.update(idhmcovid['city_ibge_code'].fillna(0))


In [153]:
idhmcovid.isnull().sum()

ANO                                              0
UF                                               0
Codmun7                                          0
Município                                        0
expec_vida                                       0
                                                ..
IDHM_L                                           0
IDHM_R                                           0
city_ibge_code                                   0
estimated_population_2019                        0
last_available_confirmed_per_100k_inhabitants    0
Length: 74, dtype: int64

**Juntando dados da mortalidade**

In [154]:
idhmcovid_wmort = pd.merge(base_mort, idhmcovid, left_on="ibgeID", right_on="city_ibge_code", how="left")

**Separando apenas as colunas necessárias para selecionar os previsores e a classe**

In [155]:
cols = list(idhmcovid.columns.values)
idhmcovid_classif = idhmcovid_wmort[['deaths_per_100k_inhabitants', 'expec_vida', 'taxa_fecund','MORT1','%sobreviver_60a','taxa_envelh','expec_anos_estudo_18','T_ANALF11A14','T_ANALF25M','T_ATRASO_0_BASICO','T_ATRASO_2_BASICO','T_FLPRE','T_FLSUPER','T_FREQSUPER1517','T_MED25M','T_SUPER25M','GINI','PIND','PINDCRI','PMPOB','PMPOBCRI','RDPC','CPR','EMP','P_AGRO','P_COM','P_CONSTR','P_EXTR','P_FORMAL','P_FUND','P_MED','P_SERV','P_SIUP','P_SUPER','T_DES2529','TRABCC','TRABPUB','TRABSC','T_AGUA','T_BANAGUA','T_DENS','T_LIXO','T_LUZ','AGUA_ESGOTO','PAREDE','T_CRIFUNDIN_TODOS','T_FORA4A5','T_FORA6A14','T_FUNDIN_TODOS','T_FUNDIN_TODOS_MMEIO','T_FUNDIN18MINF','T_M10A14CF','T_M15A17CF','T_MULCHEFEFIF014','T_NESTUDA_NTRAB_MMEIO','T_OCUPDESLOC_1','T_RMAXIDOSO','T_SLUZ','HOMEMTOT','MULHERTOT','pesoRUR','pesotot','pesourb','POP','IDHM','IDHM_E','IDHM_L','IDHM_R','estimated_population_2019', 'last_available_confirmed_per_100k_inhabitants']]
idhmcovid_classif.head()

,deaths_per_100k_inhabitants,expec_vida,taxa_fecund,MORT1,%sobreviver_60a,taxa_envelh,expec_anos_estudo_18,T_ANALF11A14,T_ANALF25M,T_ATRASO_0_BASICO,T_ATRASO_2_BASICO,T_FLPRE,T_FLSUPER,T_FREQSUPER1517,T_MED25M,T_SUPER25M,GINI,PIND,PINDCRI,PMPOB,PMPOBCRI,RDPC,CPR,EMP,P_AGRO,P_COM,P_CONSTR,P_EXTR,P_FORMAL,P_FUND,P_MED,P_SERV,P_SIUP,P_SUPER,T_DES2529,TRABCC,TRABPUB,TRABSC,T_AGUA,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE,T_CRIFUNDIN_TODOS,T_FORA4A5,T_FORA6A14,T_FUNDIN_TODOS,T_FUNDIN_TODOS_MMEIO,T_FUNDIN18MINF,T_M10A14CF,T_M15A17CF,T_MULCHEFEFIF014,T_NESTUDA_NTRAB_MMEIO,T_OCUPDESLOC_1,T_RMAXIDOSO,T_SLUZ,HOMEMTOT,MULHERTOT,pesoRUR,pesotot,pesourb,POP,IDHM,IDHM_E,IDHM_L,IDHM_R,estimated_population_2019,last_available_confirmed_per_100k_inhabitants
0,79.79027,74.82,1.98,13.4,83.07,5.73,7.94,0.97,9.96,50.47,25.44,66.21,12.69,4.37,25.01,7.23,0.42,1.97,3.68,6.18,11.61,574.96,24.49,0.73,13.47,13.92,9.57,0.10,50.54,52.98,31.97,40.51,1.16,8.14,1.88,39.40,8.65,24.94,93.06,99.01,19.52,99.83,99.33,0.46,0.05,32.03,16.71,2.84,29.83,11.35,38.25,0.00,6.76,16.46,5.77,0.98,1.68,0.67,3371.0,3505.0,1795.0,6876.0,5081.0,6854.0,0.708,0.622,0.830,0.687,8773.0,6086.85740
1,14.30820,75.35,1.97,14.8,83.02,9.77,8.68,0.92,11.05,59.83,20.05,17.63,18.55,1.21,16.28,5.96,0.47,1.85,2.83,7.94,16.70,596.18,36.21,0.20,37.90,7.97,4.09,0.17,41.97,42.63,20.54,26.86,0.19,6.87,5.05,26.56,10.22,18.65,88.50,98.18,8.62,98.03,98.55,1.26,0.53,35.53,69.74,1.69,37.08,12.10,48.72,1.11,6.50,5.40,7.05,0.18,2.24,1.45,3400.0,3304.0,2515.0,6704.0,4189.0,6658.0,0.689,0.563,0.839,0.693,6989.0,1230.50508
2,54.88474,75.46,1.66,12.6,84.00,7.13,9.31,1.04,13.80,66.18,16.70,31.87,6.79,1.11,24.56,4.92,0.43,2.00,2.89,8.45,14.93,519.87,32.61,0.48,26.03,11.83,5.76,0.32,39.37,48.83,31.03,41.76,0.55,5.38,3.54,28.32,7.05,28.55,94.50,94.70,14.99,98.49,99.68,3.82,1.13,34.35,46.03,5.65,34.83,16.10,47.41,0.00,6.06,13.72,11.04,0.12,2.81,0.32,8089.0,7668.0,4979.0,15757.0,10778.0,15089.0,0.689,0.579,0.841,0.671,20042.0,633.66929
3,71.02183,72.89,2.37,19.0,86.58,4.91,8.99,5.33,18.58,48.67,31.04,67.04,5.73,0.93,25.33,4.53,0.53,18.98,25.66,38.95,51.18,293.01,37.68,0.84,31.83,16.27,8.15,0.51,32.99,44.90,30.40,31.12,0.42,5.59,10.69,17.68,8.30,24.64,68.86,44.71,62.09,97.86,91.80,39.80,6.80,38.08,16.49,2.32,30.56,25.03,50.18,0.21,10.29,19.71,19.72,2.36,4.23,8.20,71630.0,69470.0,58102.0,141100.0,82998.0,140055.0,0.628,0.537,0.798,0.579,157698.0,2018.41495
4,8.60696,75.90,2.33,14.0,83.81,10.13,9.19,1.61,10.68,57.29,18.65,56.65,15.45,0.00,19.24,6.53,0.54,1.61,3.76,6.69,13.30,707.24,20.69,1.88,22.08,17.33,6.74,0.20,55.37,39.89,25.32,40.06,0.76,7.97,5.71,39.32,5.49,28.38,98.40,97.43,10.34,98.42,99.92,0.17,0.13,45.20,32.11,0.18,41.32,14.74,45.77,0.00,5.08,20.27,9.27,1.05,2.81,0.08,11230.0,11460.0,2986.0,22690.0,19704.0,22456.0,0.698,0.556,0.848,0.720,23237.0,223.78104


In [159]:
idhmcovid_classif.fillna(0,inplace=True)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [160]:
idhmcovid_classif.isnull().sum()

deaths_per_100k_inhabitants                      0
expec_vida                                       0
taxa_fecund                                      0
MORT1                                            0
%sobreviver_60a                                  0
                                                ..
IDHM_E                                           0
IDHM_L                                           0
IDHM_R                                           0
estimated_population_2019                        0
last_available_confirmed_per_100k_inhabitants    0
Length: 70, dtype: int64

**Lembrete para testes de exportação de CSV**

In [19]:
#idhmcovid_classif.to_csv (r'C:\Users\Jorge\Documents\TCC\idhm-pb-master\idhmcovid_class.csv', index = False, header=True)
#idhmcovid.to_csv (r'C:\Users\Jorge\Documents\TCC\idhm-pb-master\idhmcovid.csv', index = False, header=True)

**Separando previsores e classe**

In [250]:
previsores = idhmcovid_classif.iloc[:, 1:69]
classe_incid = idhmcovid_classif.iloc[:, 69].values
classe_mort = idhmcovid_classif.iloc[:, 0].values

**Transformação de atributos categóricos para numéricos**





In [ ]:
#from sklearn.preprocessing import LabelEncoder
#labelencoder = LabelEncoder()
#previsores.apply(LabelEncoder().fit_transform)

**Tratamento do dataframe de classe**

In [255]:
dataFrame = pd.DataFrame(data=classe_incid);
dataFrame.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame.head()
categorias_incid = dataFrame.niveis

In [256]:
dataFrame2 = pd.DataFrame(data=classe_mort);
dataFrame2.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame2.head()
categorias_mort = dataFrame2.niveis

In [232]:
categorias_incid = categorias_incid.values.reshape(-1, 1)
categorias_mort = categorias_mort.values.reshape(-1, 1)

**Visualização de quantos registros existem por classe**

In [233]:
unicos, quantidade = np.unique(categorias_incid, return_counts = True)
unicos

array([0.00000000e+00, 1.08436300e+01, 1.18497500e+01, ...,
       1.61887477e+04, 1.66403690e+04, 1.68099931e+04])

In [234]:
unicos2, quantidade2 = np.unique(categorias_mort, return_counts = True)
unicos2

array([0.0000000e+00, 3.2850000e-02, 6.9740000e-02, ..., 2.4958403e+02,
       2.7662517e+02, 3.3546813e+02])

**Agrupamento com k-means, utilizando 3 clusters**

In [235]:
cluster_inc = KMeans(n_clusters = 3)
cluster_inc.fit(categorias_incid)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [236]:
cluster_mort = KMeans(n_clusters = 3)
cluster_mort.fit(categorias_mort)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Visualização dos três centroides**

In [237]:
centroides_inc = cluster_inc.cluster_centers_
centroides_inc

array([[2747.40140811],
       [ 897.65401149],
       [6203.97343847]])

In [238]:
centroides_mort = cluster_mort.cluster_centers_
centroides_mort

array([[ 11.70005149],
       [106.26754856],
       [ 50.88111962]])

**Visualização dos grupos que cada registro foi associado** (Agrupamento)

In [239]:
previsoes = cluster_inc.labels_
previsoes

array([2, 1, 1, ..., 1, 0, 1], dtype=int32)

In [240]:
previsoes2 = cluster_mort.labels_
previsoes2

array([1, 0, 2, ..., 0, 1, 0], dtype=int32)

In [257]:
dataFrame = pd.DataFrame(data=previsoes);
dataFrame.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame.head()
categorias_incid = dataFrame.niveis

In [258]:
dataFrame2 = pd.DataFrame(data=previsoes2);
dataFrame2.rename(columns = {0: 'niveis'}, inplace=True)
dataFrame2.head()
categorias_mort = dataFrame2.niveis

**Contagem dos registros por classe**

In [241]:
unicos_inc, quantidade_inc = np.unique(previsoes, return_counts = True)
unicos_inc

array([0, 1, 2], dtype=int32)

In [261]:
quantidade_inc

array([1695, 3579,  315])

In [259]:
categorias_incid

0       2
1       1
2       1
3       0
4       1
       ..
5584    1
5585    1
5586    1
5587    0
5588    1
Name: niveis, Length: 5589, dtype: int32

In [245]:
unicos_mort, quantidade_mort = np.unique(previsoes, return_counts = True)
unicos_mort

array([0, 1, 2], dtype=int32)

In [262]:
quantidade_mort

array([1695, 3579,  315])

In [260]:
categorias_mort

0       1
1       0
2       2
3       2
4       0
       ..
5584    0
5585    2
5586    0
5587    1
5588    0
Name: niveis, Length: 5589, dtype: int32

**Exportar csv de previsores e classe para os classificadores**

In [263]:
previsores.to_csv (r'/content/drive/My Drive/Colab Notebooks/covid-idhm/previsores.csv', index = False, header=True)
categorias_incid.to_csv (r'/content/drive/My Drive/Colab Notebooks/covid-idhm/classe_inc.csv', index = False, header=True)
categorias_mort.to_csv (r'/content/drive/My Drive/Colab Notebooks/covid-idhm/classe_inc.csv', index = False, header=True)
